##Clustering

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TFM/Dataset y archivos iniciales/datasetSeleccionadasPreClustering.csv', decimal='.')

In [ ]:
#Eliminar promedios y columnas que no se van a utilizar
eliminacion_clumnas_menor_60 = ['id', 'home_clearances', 'home_fouls_conceded', 'home_offsides', 'home_tackles', 'away_clearances', 'away_fouls_conceded',
                                'away_offsides', 'away_tackles', 'clearances_avg_H', 'fouls_conceded_avg_H', 'offsides_avg_H', 'tackles_avg_H',
                                'fouls_conceded_avg_A', 'offsides_avg_A', 'tackles_avg_A']
df.drop(eliminacion_clumnas_menor_60, axis=1, inplace=True)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Calcular variables defensivas
df['home_goals_conceded'] = df['goal_away_ft']
df['away_goals_conceded'] = df['goal_home_ft']
df['home_shots_received'] = df['away_shots']
df['away_shots_received'] = df['home_shots']
df['home_shots_on_target_received'] = df['away_shots_on_target']
df['away_shots_on_target_received'] = df['home_shots_on_target']

# Concatenar los datos de local y visitante
teams_home = df[['home_team', 'home_goals_conceded', 'home_shots_received', 'home_shots_on_target_received', 'season']]
teams_away = df[['away_team', 'away_goals_conceded', 'away_shots_received', 'away_shots_on_target_received', 'season']].rename(columns={
    'away_team': 'home_team',
    'away_goals_conceded': 'home_goals_conceded',
    'away_shots_received': 'home_shots_received',
    'away_shots_on_target_received': 'home_shots_on_target_received'
})

teams = pd.concat([teams_home, teams_away])

# Calcular el número total de partidos jugados por cada equipo
teams['matches_played'] = 1
total_matches = teams.groupby('home_team')['matches_played'].sum().reset_index().rename(columns={'home_team': 'team', 'matches_played': 'total_matches'})

# Agrupar por equipo y sumar los valores totales
teams_grouped = teams.groupby('home_team').agg({
    'home_goals_conceded': 'sum',
    'home_shots_received': 'sum',
    'home_shots_on_target_received': 'sum',
    'season': 'nunique'
}).reset_index().rename(columns={
    'home_team': 'team',
    'home_goals_conceded': 'total_goals_conceded',
    'home_shots_received': 'total_shots_received',
    'home_shots_on_target_received': 'total_shots_on_target_received',
    'season': 'seasons'
})

teams_grouped = pd.merge(teams_grouped, total_matches, on='team')

# Calcular los promedios por partido
teams_grouped['goals_conceded_per_match'] = teams_grouped['total_goals_conceded'] / teams_grouped['total_matches']
teams_grouped['shots_received_per_match'] = teams_grouped['total_shots_received'] / teams_grouped['total_matches']
teams_grouped['shots_on_target_received_per_match'] = teams_grouped['total_shots_on_target_received'] / teams_grouped['total_matches']

# Columnas finales
teams = teams_grouped[['team', 'goals_conceded_per_match', 'shots_received_per_match', 'shots_on_target_received_per_match', 'seasons']]

scaler = StandardScaler()
scaled_defensive_data = scaler.fit_transform(teams[['goals_conceded_per_match', 'shots_received_per_match', 'shots_on_target_received_per_match']])

# KMeans con el número óptimo de clusters (3)
kmeans = KMeans(n_clusters=3, random_state=42)
teams['defensive_cluster'] = kmeans.fit_predict(scaled_defensive_data)

cluster_reassignment = {0: 2, 1: 0, 2: 1}
teams['defensive_cluster'] = teams['defensive_cluster'].map(cluster_reassignment)

#Silhouette
silhouette_avg = silhouette_score(scaled_defensive_data, teams['defensive_cluster'])

# Asignar descripciones a los clusters
cluster_descriptions = {
    0: "Defensa fuerte",
    1: "Defensa promedio",
    2: "Defensa débil"
}

teams['cluster_description'] = teams['defensive_cluster'].map(cluster_descriptions)

# Ordenar por `defensive_cluster`
teams = teams.sort_values(by='defensive_cluster').reset_index(drop=True)

output_path = '/content/drive/MyDrive/TFM/5. Rendimiento defensivo/defensive_clusters_descriptions_corrected_final.csv'
teams[['team', 'goals_conceded_per_match', 'shots_received_per_match', 'shots_on_target_received_per_match', 'seasons', 'defensive_cluster', 'cluster_description']].to_csv(output_path, index=False)

silhouette_avg, output_path


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-15-b926e694ba4e>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams['defensive_cluster'] = kmeans.fit_predict(scaled_defensive_data)
<ipython-input-15-b926e694ba4e>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teams['defensive_cluster'] = teams['defensiv

(0.4710535686062787,
 '/content/drive/MyDrive/TFM/5. Rendimiento defensivo/defensive_clusters_descriptions_corrected_final.csv')